In [1]:
import cv2
import os
import numpy as np
from cellpose import models, io

# Function to crop the image into patches
def crop_image(image, patch_size):
    height, width = image.shape[:2]
    patches = []
    for y in range(0, height, patch_size):
        for x in range(0, width, patch_size):
            patch = image[y:y+patch_size, x:x+patch_size]
            patches.append(patch)
    return patches

In [ ]:
# Load the image of the 96 well plate
plate_image = cv2.imread("D:/AI-ML TenSixty INTERNSHIP/96_well_plate.jpg")

# Define patch size (adjust according to your well dimensions)
patch_size = 100

# Specify the desired output directory
output_dir = "D:/AI-ML TenSixty INTERNSHIP/original well patches"

# Create directory to save cropped images
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Crop the image into patches
well_patches = crop_image(plate_image, patch_size)

In [ ]:
# Save each patch as a separate image file
for i, patch in enumerate(well_patches):
    output_path = os.path.join(output_dir, f"well_{i}.jpg")
    cv2.imwrite(output_path, patch)

In [ ]:
# Initialize Cellpose model for nuclei segmentation
model = models.Cellpose(gpu=False, model_type='nuclei')

# List to store segmentation results for each well
segmentation_results = []

In [ ]:
# Analyze each well using Cellpose
for i, patch_path in enumerate(well_patches):
    # Load the cropped image
    patch_image = io.imread(patch_path)

    # Segment nuclei using Cellpose
    masks, _, _, _ = model.eval(patch_image, diameter=30, channels=[0, 0])

    # Save segmentation mask
    segmentation_results.append(masks)

In [ ]:
# Convert segmentation results to binary masks
binary_masks = [(mask > 0).astype(np.uint8) for mask in segmentation_results]

In [ ]:
# Define the directory to save the segmentation mask images
mask_output_dir = "D:/AI-ML TenSixty INTERNSHIP/segmented image"

# Create the directory if it doesn't exist
os.makedirs(mask_output_dir, exist_ok=True)

# Save segmentation masks as images (optional)
for i, mask in enumerate(binary_masks):
    output_mask_path = os.path.join(mask_output_dir, f"well_{i}_mask.jpg")
    cv2.imwrite(output_mask_path, mask * 255)

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd

# Function to calculate average brightness within segmented cells
def calculate_average_brightness(segmentation_mask, well_image):
    brightness_values = []
    for cell_index in range(1, np.max(segmentation_mask) + 1):
        # Mask the original image with the segmentation mask for the current cell
        masked_image = cv2.bitwise_and(well_image, well_image, mask=np.uint8(segmentation_mask == cell_index))

        # Calculate the mean intensity of the pixels within the masked region
        brightness = np.mean(masked_image[np.nonzero(masked_image)])
        brightness_values.append(brightness)

    # Calculate the average brightness for the well
    average_brightness = np.mean(brightness_values)
    return average_brightness

In [ ]:
# Directory paths
segmentation_masks_dir = "D:/AI-ML TenSixty INTERNSHIP/segmented image"
well_images_dir = "D:/AI-ML TenSixty INTERNSHIP/original well patches"

# Initialize a list to store results
results = []

In [ ]:
# Loop through segmentation masks and original images
for filename in os.listdir(segmentation_masks_dir):
    if filename.endswith(".jpg"):
        # Load segmentation mask
        segmentation_mask_path = os.path.join(segmentation_masks_dir, filename)
        segmentation_mask = cv2.imread(segmentation_mask_path, cv2.IMREAD_GRAYSCALE)

        # Corresponding well image filename
        well_image_filename = filename.replace("_mask.jpg", ".jpg")
        well_image_path = os.path.join(well_images_dir, well_image_filename)

        # Load original well image
        well_image = cv2.imread(well_image_path, cv2.IMREAD_COLOR)

        # Calculate average brightness
        average_brightness = calculate_average_brightness(segmentation_mask, well_image)

        # Store results
        results.append({
            'Well': filename.split('_')[1].split('.')[0],  # Extract well ID from filename
            'Average_Brightness': average_brightness
        })

In [ ]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save results to CSV
results_csv_path = "D:/AI-ML TenSixty INTERNSHIP/average_brightness.csv"
results_df.to_csv(results_csv_path, index=False)